In [2]:
from requests_html import HTML
import selenium
import datetime as dt
from selenium import webdriver

In [3]:
page = 'https://nha.chotot.com/tp-ho-chi-minh/mua-ban-nha-dat?page='

In [4]:
# Hàm nhận vào link tới trang web.
# Hàm trả ra Element chứa thông tin cần thiết.
# "Vì chotot sẽ có thông tin ưu tiên và cả thông tin bình thường,...
# nên hàm sẽ trả về một tuple chứa thông tin ưu tiên trước, sau đó là thông tin bình thường"
def get_inform_from_page(link, num_page):
    ff = webdriver.Firefox()
    link = link + str(num_page)
    ff.get(link)
    # Bấm tắt thông báo hiện lên
    ff.find_element_by_css_selector('.ctTooltipClose').click()
    # Đổi giao diện để có thể thao tác dễ hơn
    ff.find_element_by_css_selector('.glyphicon.glyphicon-th-large').click()
    r = HTML(html=ff.page_source)
    tin_bt = r.find('._3AO2qzWF3LGsnNA73I-RLF')
    # tin_uu_tien = ff.find_element_by_class_name('ctAdLitingThumbnail')
    ff.close()
    return tin_bt

In [5]:
# Thuộc tính bao gồm tựa đề, giá tiền, loại hình nhà (cá nhân hay môi giới hay bất động sản), 
# link tới bài viết cụ thể, thời gian đăng tin.
#################################################################################################
# Trong các bài viết cụ thể còn có các thuộc tính khác như:
# - Diện tích
# - Hướng cửa chính
# - Số phòng vệ sinh
# - Số tầng
# - Loại hình nhà ở
# - Giấy tờ pháp lý
# - Đặc điểm nhà đất
# - Địa chỉ cụ thể

In [6]:
title1_list = []
price_list = []
title2_list = []
type1_list = []
time_list = []
link_list = []

In [7]:
link_prefit = 'https://nha.chotot.com/'
for ind in range(500,1000):
    try:
        thong_thuong = get_inform_from_page(page, ind)
        n = len(thong_thuong)
        for i in range(n):
            # Lấy link dắt tới thông tin cụ thể
            link = thong_thuong[i].find('a', first=True).attrs['href']
            link = link_prefit + link
            try:
                #Title 1 là title của bài viết.
                title1 = thong_thuong[i].find('h4', first=True).text
                #Price là giá tiền của căn nhà đó.
                price = thong_thuong[i].find('p', first=True).text
                #Title 2 là title của ảnh.
                title2 = thong_thuong[i].find('img')[0].attrs['alt']
                # Type là loại hình mua bán, cá nhân, môi giới??
                type1 = thong_thuong[i].find('img')[1].attrs['alt']
                # Time sẽ là thời gian đăng tin, Time ở trên trang web sẽ ghi là 1 ngày trước, 1 phút trước, ...
                # Nên ta sẽ trích thời gian hiện tại và trừ ra khoảng thời gian đó để có được thời gian đăng tin.
                time = dt.datetime.now()
                time_text = thong_thuong[i].find('span')
                time_text = time_text[-1].text
                try:
                    diff = int(time_text.split(' ')[0])
                    if 'phút' in time_text.lower():
                        time = time - dt.timedelta(minutes=diff)
                    elif 'giờ' in time_text.lower():
                        time = time - dt.timedelta(hours=diff)
                    elif 'ngày' in time_text.lower():
                        time = time - dt.timedelta(date=diff)
                    elif 'tháng' in time_text.lower():
                        time = time - dt.timedelta(months=diff)
                except:
                    time = None

                title1_list.append(title1)
                title2_list.append(title2)x
                price_list.append(price)
                type1_list.append(type1)
                time_list.append(time)
                link_list.append(link)
            except:
                print(link)
    except:
        print(ind)

https://nha.chotot.com//tp-ho-chi-minh/quan-binh-thanh/mua-ban-nha-dat/65205571.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-12/mua-ban-nha-dat/65180994.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-11/mua-ban-nha-dat/65154888.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-phu-nhuan/mua-ban-nha-dat/65152444.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-12/mua-ban-nha-dat/64972427.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-go-vap/mua-ban-nha-dat/65141140.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-thu-duc/mua-ban-nha-dat/65132119.htm
https://nha.chotot.com//tp-ho-chi-minh/huyen-hoc-mon/mua-ban-nha-dat/65120432.htm
https://nha.chotot.com//tp-ho-chi-minh/huyen-hoc-mon/mua-ban-nha-dat/65113606.htm
https://nha.chotot.com//tp-ho-chi-minh/huyen-binh-chanh/mua-ban-nha-dat/65100494.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-3/mua-ban-nha-dat/65099099.htm
https://nha.chotot.com//tp-ho-chi-minh/quan-thu-duc/mua-ban-nha-dat/65092091.htm
https://nha.chotot.com//tp-ho-chi-minh/

In [8]:
print(len(link_list))
import pandas as pd
data = pd.DataFrame({'title1':title1_list, 'title2':title2_list, 'price': price_list, 'type':type1_list, 'time': time_list, 'link':link_list})
data.to_csv('data-full.csv', index=None)

9978
